# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: fetch latest code</h2>
            <span style="color:#f71;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use GPT-4o and Claude-3.5-Sonnet, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please make the suggested switches to the models (3 cells down from here).
            </span>
        </td>
    </tr>
</table>

In [1]:
#! pip install gradio

# imports

#import os
#import io
#import sys
#from dotenv import load_dotenv
#from openai import OpenAI
#import google.generativeai
#import anthropic
#from IPython.display import Markdown, display, update_display
#import gradio as gr
#import subprocess

In [2]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [3]:
print(system_message)

You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. The C++ response needs to produce an identical output in the fastest possible time.


In [4]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [5]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [6]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [7]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [8]:
exec(pi)

Result: 3.141592658589
Execution Time: 22.617263 seconds


In [9]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [10]:
!pip install -q  transformers bitsandbytes
from google.colab import userdata
from huggingface_hub import login

from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# HF
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)


In [11]:

# Qwen2.5-Coder-32B-Instruct

#model_name = "Qwen/Qwen2.5-Coder-14B-Instruct"
#model_name = "Qwen/Qwen2.5-Coder-32B-Instruct"
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
#model_name = "Qwen/CodeQwen1.5-7B-Chat"
#model_name = "Qwen/Qwen2.5-Coder-14B-Instruct-GGUF"


def create_model_and_tokenizer():

    # Configure 4-bit quantization
    bnb_config_4bit = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",  # normalized float 4
        bnb_4bit_use_double_quant=True
    )

    # Configure 8-bit quantization
    bnb_config_8bit = BitsAndBytesConfig(
      load_in_8bit=True,
      llm_int8_enable_fp32_cpu_offload=True,
      llm_int8_threshold=6.0, # 10.0,
      llm_int8_skip_modules=["lm_head"],
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto",
#        quantization_config=bnb_config_4bit, # <------------------------------ Quant ON/OFF
        quantization_config=bnb_config_8bit
        )

    print("---------------------------------------------------")
    memory = model.get_memory_footprint() / 1e6
    print(f"Model Get Memory footprint is: =====> {memory:,.1f} MB")
    model

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer


In [12]:


import torch
import gc
import os

def cleanup_model_resources():
    """
    Comprehensive cleanup of model resources and memory
    """
    # Clear CUDA cache if using GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        # Optional: Reset the CUDA device
        torch.cuda.reset_peak_memory_stats()
        # Optional: Synchronize CUDA to ensure all operations are complete
        torch.cuda.synchronize()

    # Run garbage collection multiple times to catch circular references
    for _ in range(3):
        gc.collect()

    # Force Python to release memory back to OS (Unix/Linux only)
    if hasattr(os, 'malloc_trim'):
        os.malloc_trim(0)

    if 'model' in locals():

        # Clear the model from GPU memory explicitly
        if hasattr(model, 'cpu'):
            model.cpu()

        # Clear specific components
        if hasattr(model, 'transformer'):
            del model.transformer
        if hasattr(model, 'embeddings'):
            del model.embeddings

        del model

In [13]:
def run(system_message: str, user_prompt: str, model, tokenizer, max_new_tokens: int = 512) -> str:
    """
    Generate a response using a chat model.

    Args:
        system_message (str): The system message to set context
        user_prompt (str): The user's input prompt
        model: The loaded model instance
        tokenizer: The loaded tokenizer instance
        max_new_tokens (int): Maximum number of tokens to generate

    Returns:
        str: The generated response
    """
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ]


        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
#            **model_inputs,
            model_inputs.input_ids,  # NEW for Qwen/coder
            max_new_tokens=max_new_tokens
        )

        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return response

    except Exception as e:
        raise Exception(f"Error in text generation: {str(e)}")
    finally:
        # Clean up any tensors to help with memory management
        if 'model_inputs' in locals():
            del model_inputs
        if 'generated_ids' in locals():
            del generated_ids
        torch.cuda.empty_cache()

# Example usage:
# response = run(
#     system_message="You are a helpful assistant",
#     user_prompt="Write a hello world program",
#     model=model,
#     tokenizer=tokenizer
# )

In [ ]:
## STEP 4:

model, tokenizer = create_model_and_tokenizer()


#cleanup_model_resources()

In [ ]:
python = python_hard

system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

# possibly give hints, like:
## Kadane's algorithm for max subarray sum
algoritm_hint = "Your code does not need to follow the same algorithms, so consider using other relevant algorithms in the same problem domain."
#algoritm_hint += "If you can use better algorithms please do so, and add a small comment on the algorithm used. "

user_msg = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
user_msg += "Respond only with C++ code; do not explain your work other than a few comments. "
user_msg += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"

# Ed Hint!!
user_msg += "Keep implementations for random numbers identical so that results match exactly.\n\n"
user_msg += "# You are allowed to change the algorithms if there are suitable optimizations. The only requirement is that the final result is the same.\n\n"
#user_msg += "Be careful of errors like: shift count >= width of type [-Wshift-count-overflow]unsigned int lcg(unsigned int& seed, unsigned int a = 1664525, unsigned int c = 1013904223, unsigned int m = 1 << 32)"
#user_msg += "Be careful to support large number sizes"





# apply hints
#system_message += algoritm_hint
#user_msg += algoritm_hint

print (system_message)
print (user_msg)
print (python)


In [17]:


def enriched_user_prompt_for(msg, python):
    user_prompt = msg
    user_prompt += python
    return user_prompt


response = run(
  system_message=system_message,
  user_prompt=enriched_user_prompt_for(user_msg, python),
  model=model,
  tokenizer=tokenizer,
  max_new_tokens=6048 #1024
)

print (response)
write_output(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Okay, I need to reimplement the given Python code into high-performance C++. Let me start by understanding what each part does.

First, the Python code defines an LCG (Linear Congruential Generator) using a generator function. Then, it uses this to generate random numbers for the maximum subarray sum problem, applying Kadane's algorithm. Finally, it runs this 20 times with different seeds and sums the results.

So, in C++, I'll need to replicate the LCG. But wait, in Python, the modulus is applied after each yield. In C++, I should use 64-bit integers to prevent overflow since the a and c values are large. So, I'll define the LCG state as a uint64_t.

Next, the max_subarray_sum function. The Python version uses a nested loop, which is O(n^2). For n=10000, that's 100 million operations. In C++, I can optimize this by using Kadane's algorithm, which is O(n). That's a big improvement.

Wait, the Python code uses a nested loop to compute the maximum subarray sum. But Kadane's algorithm is 

In [31]:
cleanup_model_resources()

# Compiling C++ and executing

This next cell contains the command to compile a C++ file on my M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In the next lab (day4), a student has contributed a full solution that compiles to efficient code on Mac, PC and Linux!

You can wait for this, or you can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below.
If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.


OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [ ]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
exec(python_hard)

In [ ]:
optimize_gpt(python_hard)

In [ ]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
# T4:
# For compiling C++ code optimized for a NVIDIA T4 GPU,
# you'll want to use NVIDIA's CUDA compiler (nvcc)

!nvcc -O3 -std=c++17 -arch=sm_75 -o optimized optimized.cpp
!./optimized

#Key points about these flags:

# -arch=sm_75 is specifically for T4 GPUs (which use the Turing architecture with compute capability 7.5)
# -O3 enables high-level optimizations
# -std=c++17 enables C++17 features

# CPU:
# If your code doesn't actually use CUDA/GPU features and
# is just running on CPU, you could use g++ instead:
#!g++ -O3 -std=c++17 -march=native -mtune=native -o optimized optimized.cpp
#!./optimized


In [ ]:
optimize_claude(python_hard)

In [ ]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M1 Mac version to compile and execute optimized C++ code:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)